# Install package

In [12]:
# !pip install transformers
# !pip install openai
# !pip install streamlit
# !pip install pandas
# !pip install torch

# Generate Data

In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

# User ID range
user_ids = [1, 2, 3, 4, 5]

# Food and nutrition data
foods = [
    {"name": "Apple", "calories": 52, "protein": 0.3, "carbs": 14, "fats": 0.2, "fiber": 2.4},
    {"name": "Banana", "calories": 96, "protein": 1.3, "carbs": 27, "fats": 0.3, "fiber": 2.6},
    {"name": "Chicken Breast", "calories": 165, "protein": 31, "carbs": 0, "fats": 3.6, "fiber": 0},
    {"name": "Broccoli", "calories": 55, "protein": 3.7, "carbs": 11.2, "fats": 0.6, "fiber": 3.8},
    {"name": "Rice", "calories": 130, "protein": 2.7, "carbs": 28, "fats": 0.3, "fiber": 0.4},
]

# User information
user_info = {
    1: {"age": 25, "gender": "Male", "personality": "Active", "preferences": "Low fat", "routine": "Morning exercise"},
    2: {"age": 30, "gender": "Female", "personality": "Sedentary", "preferences": "High protein", "routine": "Evening walks"},
    3: {"age": 35, "gender": "Male", "personality": "Moderately active", "preferences": "Balanced diet", "routine": "Afternoon gym"},
    4: {"age": 40, "gender": "Female", "personality": "Active", "preferences": "Low carb", "routine": "Morning yoga"},
    5: {"age": 28, "gender": "Male", "personality": "Sedentary", "preferences": "High fiber", "routine": "No specific routine"},
}

# Generate 30 days of data
dates = [(datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(30)]

# Nutrition data
nutrition_records = []
health_records = []

for user_id in user_ids:
    for date in dates:
        daily_calories = 0
        daily_protein = 0
        daily_carbs = 0
        daily_fats = 0
        daily_fiber = 0
        daily_foods = []
        
        # Randomly select 3-5 types of food
        daily_food_count = random.randint(3, 5)
        for _ in range(daily_food_count):
            food = random.choice(foods)
            daily_foods.append(food["name"])
            daily_calories += food["calories"]
            daily_protein += food["protein"]
            daily_carbs += food["carbs"]
            daily_fats += food["fats"]
            daily_fiber += food["fiber"]
        
        nutrition_records.append({
            "user_id": user_id,
            "date": date,
            "foods": ", ".join(daily_foods),
            "calories": daily_calories,
            "protein": daily_protein,
            "carbs": daily_carbs,
            "fats": daily_fats,
            "fiber": daily_fiber
        })

        # Generate health data
        health_records.append({
            "user_id": user_id,
            "date": date,
            "weight": random.uniform(60, 90),
            "cholesterol_level": random.randint(150, 250),
            "blood_pressure": f"{random.randint(110, 140)}/{random.randint(70, 90)}",
            "age": user_info[user_id]["age"],
            "gender": user_info[user_id]["gender"],
            "personality": user_info[user_id]["personality"],
            "preferences": user_info[user_id]["preferences"],
            "routine": user_info[user_id]["routine"]
        })

nutrition_df = pd.DataFrame(nutrition_records)
health_df = pd.DataFrame(health_records)

nutrition_df.to_csv('nutrition_data.csv', index=False)
health_df.to_csv('health_data.csv', index=False)

print("Generated 'nutrition_data.csv' and 'health_data.csv'")


Generated 'nutrition_data.csv' and 'health_data.csv'


# Data preprocess

In [2]:
import pandas as pd

# Data preprocessing
nutrition_df = pd.read_csv('nutrition_data.csv')
health_df = pd.read_csv('health_data.csv')

nutrition_df.fillna(0, inplace=True)
health_df.fillna(0, inplace=True)

# Merge datasets
data = pd.merge(nutrition_df, health_df, on=['user_id', 'date'])
data.to_csv('preprocessed_data.csv', index=False)

print("Data preprocessing completed and saved to 'preprocessed_data.csv'")


Data preprocessing completed and saved to 'preprocessed_data.csv'


# Interaction

In [5]:
import pandas as pd
import torch
from transformers import BertTokenizer
from openai import OpenAI


# Initialize OpenAI client
client = OpenAI(
  base_url="https://integrate.api.nvidia.com/v1",
  api_key=""
)

# Read preprocessed data
data = pd.read_csv('preprocessed_data.csv')

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Define interaction function
def interact():
    user_id = int(input("Enter your user ID (1-5): "))
    user_goal = input("Enter your health goal: ")

    # Extract user health data
    user_data = data[data['user_id'] == user_id].tail(7)  # Get the most recent 7 days of data
    user_info_data = user_data.iloc[0].to_dict()

    health_summary = f"Age: {user_info_data['age']}, Gender: {user_info_data['gender']}, Personality: {user_info_data['personality']}, Preferences: {user_info_data['preferences']}, Routine: {user_info_data['routine']}\n"
    health_details = ""
    for index, row in user_data.iterrows():
        health_details += f"Date: {row['date']}, Foods: {row['foods']}, Calories: {row['calories']}, Protein: {row['protein']}, Carbs: {row['carbs']}, Fats: {row['fats']}, Fiber: {row['fiber']}, Weight: {row['weight']:.2f}, Cholesterol Level: {row['cholesterol_level']}, Blood Pressure: {row['blood_pressure']}\n"

    prompt = f"User's recent health status: {health_summary}Based on the following health data for the past week: {health_details}, provide a personalized recommendation to achieve the goal: {user_goal}"

    completion = client.chat.completions.create(
      model="meta/llama3-70b-instruct",
      messages=[{"role":"user","content":prompt}],
      temperature=0.5,
      top_p=1,
      max_tokens=1024,
      stream=True
    )

    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")


if __name__ == "__main__":
    interact()


Enter your user ID (1-5):  1
Enter your health goal:  locker


Based on your health data for the past week, here's a personalized recommendation to help you achieve your goal:

**Congratulations on your active lifestyle and morning exercise routine!**

**Nutrition Analysis:**

* Your calorie intake varies significantly from day to day, ranging from 247 to 567 calories. Aim for a consistent daily calorie intake of 350-400 calories to support your active lifestyle.
* Your protein intake is generally high, which is excellent for muscle growth and repair. However, it's essential to balance your protein intake with complex carbohydrates and healthy fats.
* Your carbohydrate intake is moderate, but you may benefit from increasing your complex carbohydrate sources, such as whole grains, fruits, and vegetables.
* Your fat intake is relatively low, which is in line with your preference for low-fat foods. However, ensure you're getting enough healthy fats from sources like nuts, seeds, and avocados.
* Your fiber intake is excellent, with an average of 7-8 g

In [4]:
import pandas as pd
import torch
from transformers import BertTokenizer
from openai import OpenAI


# Initialize OpenAI client
client = OpenAI(
  base_url="https://integrate.api.nvidia.com/v1",
  api_key=""
)

# Read preprocessed data
data = pd.read_csv('preprocessed_data.csv')

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Define interaction function
def interact():
    user_id = int(input("Enter your user ID (1-5): "))
    user_goal = input("Enter your health goal: ")

    # Extract user health data
    user_data = data[data['user_id'] == user_id].tail(7)  # Get the most recent 7 days of data
    user_info_data = user_data.iloc[0].to_dict()

    health_summary = f"Age: {user_info_data['age']}, Gender: {user_info_data['gender']}, Personality: {user_info_data['personality']}, Preferences: {user_info_data['preferences']}, Routine: {user_info_data['routine']}\n"
    health_details = ""
    for index, row in user_data.iterrows():
        health_details += f"Date: {row['date']}, Foods: {row['foods']}, Calories: {row['calories']}, Protein: {row['protein']}, Carbs: {row['carbs']}, Fats: {row['fats']}, Fiber: {row['fiber']}, Weight: {row['weight']:.2f}, Cholesterol Level: {row['cholesterol_level']}, Blood Pressure: {row['blood_pressure']}\n"

    prompt = f"User's recent health status: {health_summary}Based on the following health data for the past week: {health_details}, provide a personalized recommendation to achieve the goal: {user_goal}"

    completion = client.chat.completions.create(
      model="meta/llama3-70b-instruct",
      messages=[{"role":"user","content":prompt}],
      temperature=0.5,
      top_p=1,
      max_tokens=1024,
      stream=True
    )

    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")


if __name__ == "__main__":
    interact()


Enter your user ID (1-5):  2
Enter your health goal:  Professional breakdancer learn all the powermoves


What a fascinating goal! As a sedentary individual, you'll need to make significant changes to your diet and lifestyle to achieve your goal of becoming a professional breakdancer. Here's a personalized recommendation to help you get started:

**Dietary Recommendations:**

1. **Increase calorie intake:** As a breakdancer, you'll need more energy to perform complex moves. Aim for an additional 500-750 calories per day to support your increased physical activity.
2. **Protein-rich foods:** Focus on high-protein foods like chicken breast, fish, eggs, and legumes to help build and repair muscles. Aim for 1.6-2.2 grams of protein per kilogram of body weight.
3. **Complex carbohydrates:** Include complex carbs like whole grains, brown rice, and vegetables to provide sustained energy. Aim for 2-3 grams of complex carbs per kilogram of body weight.
4. **Healthy fats:** Nuts, seeds, and avocados are great sources of healthy fats that support energy production and overall health.
5. **Hydration:*

In [5]:
import pandas as pd
import torch
from transformers import BertTokenizer
from openai import OpenAI


# Initialize OpenAI client
client = OpenAI(
  base_url="https://integrate.api.nvidia.com/v1",
  api_key=""
)

# Read preprocessed data
data = pd.read_csv('preprocessed_data.csv')

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Define interaction function
def interact():
    user_id = int(input("Enter your user ID (1-5): "))
    user_goal = input("Enter your health goal: ")

    # Extract user health data
    user_data = data[data['user_id'] == user_id].tail(7)  # Get the most recent 7 days of data
    user_info_data = user_data.iloc[0].to_dict()

    health_summary = f"Age: {user_info_data['age']}, Gender: {user_info_data['gender']}, Personality: {user_info_data['personality']}, Preferences: {user_info_data['preferences']}, Routine: {user_info_data['routine']}\n"
    health_details = ""
    for index, row in user_data.iterrows():
        health_details += f"Date: {row['date']}, Foods: {row['foods']}, Calories: {row['calories']}, Protein: {row['protein']}, Carbs: {row['carbs']}, Fats: {row['fats']}, Fiber: {row['fiber']}, Weight: {row['weight']:.2f}, Cholesterol Level: {row['cholesterol_level']}, Blood Pressure: {row['blood_pressure']}\n"

    prompt = f"User's recent health status: {health_summary}Based on the following health data for the past week: {health_details}, provide a personalized recommendation to achieve the goal: {user_goal}"

    completion = client.chat.completions.create(
      model="meta/llama3-70b-instruct",
      messages=[{"role":"user","content":prompt}],
      temperature=0.5,
      top_p=1,
      max_tokens=1024,
      stream=True
    )

    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")


if __name__ == "__main__":
    interact()


Enter your user ID (1-5):  1
Enter your health goal:  BBOY


What's good, young BBOY!

First, let's break down your recent health data:

**Positive trends:**

1. Your protein intake is generally high, which is great for muscle growth and maintenance.
2. You're consuming a good amount of fiber, which can help with digestion and satiety.
3. Your blood pressure is within a healthy range, and your cholesterol levels are mostly under control.

**Areas for improvement:**

1. Your carbohydrate intake is quite high, which might be affecting your weight and energy levels.
2. Your fat intake is relatively low, which could impact your energy levels and overall nutrition.
3. There's a significant variation in your weight over the past week, which might indicate that you need to focus on more balanced eating habits.

**Personalized recommendations to achieve your BBOY goal:**

1. **Balance your macronutrients:** Aim for a daily intake of 25-30% protein, 40-45% carbohydrates, and 25-30% fat. This will help you maintain energy levels and support muscle growth.

In [6]:
import pandas as pd
import torch
from transformers import BertTokenizer
from openai import OpenAI


# Initialize OpenAI client
client = OpenAI(
  base_url="https://integrate.api.nvidia.com/v1",
  api_key=""
)

# Read preprocessed data
data = pd.read_csv('preprocessed_data.csv')

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Define interaction function
def interact():
    user_id = int(input("Enter your user ID (1-5): "))
    user_goal = input("Enter your health goal: ")

    # Extract user health data
    user_data = data[data['user_id'] == user_id].tail(7)  # Get the most recent 7 days of data
    user_info_data = user_data.iloc[0].to_dict()

    health_summary = f"Age: {user_info_data['age']}, Gender: {user_info_data['gender']}, Personality: {user_info_data['personality']}, Preferences: {user_info_data['preferences']}, Routine: {user_info_data['routine']}\n"
    health_details = ""
    for index, row in user_data.iterrows():
        health_details += f"Date: {row['date']}, Foods: {row['foods']}, Calories: {row['calories']}, Protein: {row['protein']}, Carbs: {row['carbs']}, Fats: {row['fats']}, Fiber: {row['fiber']}, Weight: {row['weight']:.2f}, Cholesterol Level: {row['cholesterol_level']}, Blood Pressure: {row['blood_pressure']}\n"

    prompt = f"User's recent health status: {health_summary}Based on the following health data for the past week: {health_details}, provide a personalized recommendation to achieve the goal: {user_goal}"

    completion = client.chat.completions.create(
      model="meta/llama3-70b-instruct",
      messages=[{"role":"user","content":prompt}],
      temperature=0.5,
      top_p=1,
      max_tokens=1024,
      stream=True
    )

    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")


if __name__ == "__main__":
    interact()


Enter your user ID (1-5):  3
Enter your health goal:  PROFESSINAL MARATHON RUNNER


Based on your recent health data, I'll provide a personalized recommendation to help you achieve your goal of becoming a professional marathon runner.

**Current Status:**

* Your diet seems to be balanced, with a good mix of carbohydrates, protein, and fiber. However, there are some inconsistencies in your calorie intake, which may affect your energy levels and weight.
* Your weight has been fluctuating significantly over the past week, which may indicate that you need to focus on a more consistent and tailored nutrition plan.
* Your cholesterol levels are slightly elevated, which may be a concern for cardiovascular health. As a marathon runner, it's essential to maintain healthy cholesterol levels to reduce the risk of cardiovascular disease.
* Your blood pressure is generally within a healthy range, but there are some variations that may be related to your diet and hydration levels.

**Recommendations:**

1. **Consistent Calorie Intake:** To maintain a stable weight and energy level